# Further model predictions
For figures 4 and 5, use the model to make a few predictions to motivate the experiments done in each case. 

Figure 4: vary CD19 dose
    
Figure 5: show that HHAT is an antagonist and p8F, an agonist, based on their dose response curves or EC50s. 
Just identify two dots on an antagonism curve vs tau at the approx. tau inferred by eyeball comparison to OT-1 EC50s. Later on, could fit model $\tau$ of each antigen on the dose response data, and use that to predict antagonism properties. But now just a rough comparison to OT-1 EC50s is going to do the trick. 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import json, h5py
import os
import scipy as sp
from scipy import stats, optimize
import statsmodels.api as statsmodels
import itertools

In [ ]:
from models.tcr_car_akpr_model import (
    activation_function, steady_akpr_i_receptor_types, steady_akpr_i_1ligand, psi_of_i_gamma, 
)
from models.akpr_i_model import steady_akpr_i_2ligands
from utils.preprocess import (
    michaelis_menten, loglog_michaelis_menten, inverse_michaelis_menten, 
    string_to_tuple
)
from models.akpr_i_model import psi_of_i
from mcmc.plotting import handles_properties_legend
from mcmc.costs_tcr_car_antagonism import repackage_tcr_car_params
from mcmc.mcmc_analysis import find_best_grid_point
from mcmc.prediction_utilities_tcr_car import find_1itam_car_ampli
from utils.export import dict_to_hdf5
from utils.preprocess import geo_mean

In [ ]:
# Display figures larger
plt.rcParams['figure.dpi'] = 150 # default for me was 75
fig_dir = "figures/extra_predictions/"

#plt.rcParams['font.family'] = 'Helvetica'
# TODO: use font manager API: https://matplotlib.org/stable/api/font_manager_api.html
# to import Helvetica.ttc from the data/ folder? So it's robust on Google Drive too. 
# Not easy to get Helvetica.ttf which matplotlib would support (doesn't seem very legit). 
plt.rcParams["font.family"] = "Arial"

## Model parameters
Fitted values on TCR-TCR antagonism, then CAR-TCR antagonism. 

*Use $(k, m, f) = (1, 2, 1)$*

In [ ]:
with open(os.path.join("data", "pep_tau_map_ot1.json"), "r") as handle:
    pep_tau_map = json.load(handle)

In [ ]:
# Load best parameter fit for (1, 1, 2)
fit_conc = ["1uM", "1nM"]
analysis_res_fname = "mcmc_analysis_tcr_car_both_conc.json"
with open(os.path.join("results", "mcmc", analysis_res_fname), "r") as jfile:
    all_results_dicts = json.load(jfile)
    del jfile

# Go back to linear-scale parameters
chosen_kmf = (1, 2, 1)
pvec_best = np.asarray(all_results_dicts.get(str(chosen_kmf)).get("param_estimates").get("MAP best"))

# Load constant parameter values
samples_fname = samples_fname = "mcmc_results_tcr_car_both_conc.h5"
with h5py.File(os.path.join("results", "mcmc", samples_fname), "r") as rfile:
    data_group = rfile.get("data")
    fit_param_names = list(rfile.get("samples").attrs.get("param_names"))
    l_conc_mm_params = data_group.get("l_conc_mm_params")[()]
    cost_args_loaded = [data_group.get(a)[()]
                        for a in data_group.attrs.get("cost_args_names")]
    del data_group, rfile

In [ ]:
### Rearrange loaded parameters in the correct format 
other_rates, ritot, nmf_fixed, cd19_tau_l = cost_args_loaded
res = repackage_tcr_car_params(pvec_best, chosen_kmf, *cost_args_loaded[:3])
(
all_rates, tcr_rates, car_rates, ritot_vec, tcr_ri, car_ri,
nmf_both, tcr_nmf, car_nmf, threshold_taus
) = res
# Compute thresholds
tcr_thresh = steady_akpr_i_1ligand(tcr_rates, threshold_taus[0],
        10*tcr_ri[0], tcr_ri, tcr_nmf, large_l=True)[tcr_nmf[0]]
car_thresh = steady_akpr_i_1ligand(car_rates, threshold_taus[1],
        10*car_ri[0], car_ri, car_nmf, large_l=True)[car_nmf[0]]

# Model predictions for varying CD19 density
Predictions for the regime relevant to Nalm6 tumors with varying amounts of CD19 on their surface. For now, eyeball the data in the bar graph of figure 4. 

In [ ]:
def compute_model_output_curve(tau_tcr, l_tcr, cd19, kw_params, highlights=None, 
                               baseline=0.0, scale_down_zt=1.0):
    """ For one given CD19 condition, typically one L_TCR, and an axis of tau_T values. """
    # Prepare L, tau ranges
    if highlights is not None:
        tau_tcr2 = np.unique(np.sort(np.concatenate([tau_tcr, highlights])))
    else:
        tau_tcr2 = tau_tcr
    l_tcr2 = np.asarray(l_tcr)
    # Tonic signaling of CAR in the absence of CD19
    #if cd19[1] == 0.0:
    #    cd19 = [cd19[0]*0.01, 1e4]
    
    # Extract parameters
    p = kw_params
    
    # Agonist alone output
    ag_alone = steady_akpr_i_1ligand(p["car_rates"], *cd19, p["car_ri"], p["car_nmf"])[p["car_nmf"][0]]
    ag_alone = activation_function(ag_alone, p["car_thresh"]) + baseline
    
    # Tonic TCR signaling for CD19 alone?
    #taus = np.asarray([1.0, cd19[0]])
    #lvec = np.asarray([1e4, 0.5])
    #ag_alone = steady_akpr_i_receptor_types(p["all_rates"], taus, lvec, p["ritot_vec"], p["nmf_both"])
    #ag_alone = (activation_function(ag_alone[0][p["tcr_nmf"][0]], p["tcr_thresh"]) 
    #            + activation_function(ag_alone[1][p["car_nmf"][0]], p["car_thresh"]) + baseline)

    model_columns = pd.Index([r"$T_{}$".format(n) for n in range(p["tcr_nmf"][0]+1)] 
                       + [r"$C_{}$".format(n) for n in range(p["car_nmf"][0]+1)]
                       + [r"$S_T$", r"$S_C$", r"$Z_T$", r"$Z_C$", "Ratio"], name="Variable")
    model_index = pd.MultiIndex.from_product([l_tcr2, tau_tcr2], names=[r"$L_T$", r"$\tau_T$"])
    df_m = pd.DataFrame(np.zeros([len(model_index), len(model_columns)]), 
                           columns=model_columns, index=model_index)

    for l_t, tau_t in model_index:
        taus = np.asarray([tau_t, cd19[0]])
        lvec = np.asarray([l_t, cd19[1]])
        complexes_mix = steady_akpr_i_receptor_types(p["all_rates"], taus, lvec, p["ritot_vec"], p["nmf_both"])
        df_m.loc[(l_t, tau_t), r"$T_0$":r"$T_{}$".format(p["tcr_nmf"][0])] = complexes_mix[0]
        df_m.loc[(l_t, tau_t), r"$C_0$":r"$C_{}$".format(p["car_nmf"][0])] = complexes_mix[1]
        df_m.loc[(l_t, tau_t), r"$S_T$":r"$S_C$"] = complexes_mix[2]
        # Normalize outputs to compare CAR and TCR properly, accounting for
        # their very different signaling potencies.
        
        df_m.loc[(l_t, tau_t), r"$Z_T$"] = activation_function(complexes_mix[0][-1], p["tcr_thresh"])
        # Somehow the TCR output doesn't contribute as much; scale it down?
        df_m.loc[(l_t, tau_t), r"$Z_T$"] *= scale_down_zt
        df_m.loc[(l_t, tau_t), r"$Z_C$"] = activation_function(complexes_mix[1][-1], p["car_thresh"])
        df_m.loc[(l_t, tau_t), "Ratio"] = (df_m.loc[(l_t, tau_t), r"$Z_T$":r"$Z_C$"].sum() + baseline) / ag_alone
    
    return df_m

In [ ]:
# Tumor parameters
size_factor = 2.5  # Taisuke: B16:10, Nalm6: 5, E2aPBX: 2

molec_counts_filename = "data/surface_counts/surface_molecule_summary_stats.h5"
molec_stats = pd.read_hdf(molec_counts_filename, key="surface_numbers_stats")
mhc_levels_nalm6 = {}
cd19_levels_nalm6 = {}
tumor_order = ["CD19_High", "CD19_Med", "CD19_Low", "CD19_KO"]
tumor_names_df = ["Nalm6_19hi", "Nalm6_19int", "Nalm6_19low", "Nalm6_19KO"]
mtc = "Geometric mean"
for tumor, lbl in zip(tumor_order, tumor_names_df):
    mhc_levels_nalm6[tumor] = molec_stats.at[(lbl, "MHC"), mtc] / size_factor
    cd19_levels_nalm6[tumor] = molec_stats.at[(lbl, "CD19"), mtc] / size_factor
mhc_levels_nalm6 = geo_mean(pd.Series(mhc_levels_nalm6), axis=0)
# Try KO = 0 CD19 too for comparison

In [ ]:
# The baseline signaling is the level of activity produced by 9C alone, with CD19-KO tumor
# So that's output for a TCR only 
with open("data/pep_tau_map_others.json", "r") as h:
    other_taus = json.load(h)
# 9C doesn't give activation, 9V gives some. Take a tau halfway
tau_base = (other_taus["NYESO-9C"] + other_taus["NYESO-9V"]) / 2.0
baseline_out = activation_function(steady_akpr_i_1ligand(tcr_rates, tau_base, 
                            mhc_levels_nalm6, tcr_ri, tcr_nmf)[-2], tcr_thresh)

In [ ]:
# Nice big parameter dictionary
param_dict = {
    "car_rates": car_rates, 
    "car_ri": car_ri, 
    "car_nmf": car_nmf, 
    "all_rates": all_rates, 
    "tcr_nmf": tcr_nmf, 
    "ritot_vec": ritot_vec, 
    "nmf_both": nmf_both, 
    "tcr_thresh": tcr_thresh, # TCR threshold seems higher for PLCgamma? 
    "car_thresh": car_thresh / 20.0
}

# Compute model output for B16
tau_tcr_range = np.linspace(0.001, 4.0, 101)
l_tcr_range = np.asarray([mhc_levels_nalm6])

# Loop over tumor cell lines, concat DataFrames after
df_model = pd.concat({t: compute_model_output_curve(tau_tcr_range, l_tcr_range, 
                (cd19_tau_l[0], cd19_levels_nalm6[t]), param_dict, baseline=baseline_out) 
                for t in cd19_levels_nalm6.keys()}, 
                names=["Tumor"], axis=0)


In [ ]:
# Plot antagonism ratio vs tau for the L_TCR, L_CAR conditions of B16 tumors. 
# Color palette for the tumors: mako
tumor_colors = sns.color_palette("viridis_r", n_colors=len(cd19_levels_nalm6))
tumor_colors = {t:tumor_colors[i] for i, t in enumerate(tumor_order)}

fig, ax = plt.subplots()
fig.set_size_inches(2.75, 2.25)
xvals = np.sort(df_model.index.get_level_values(r"$\tau_T$").unique().values)

tumor_styles = ["-", "--", ":", "-."]
tumor_styles = {t:tumor_styles[i] for i, t in enumerate(tumor_order)}

nice_tumor_lbls = {
    "CD19_High":r"CD19${}^{\mathrm{Hi}}$", 
    "CD19_Med":r"CD19${}^{\mathrm{Int}}$", 
    "CD19_Low":r"CD19${}^{\mathrm{Lo}}$",
    "CD19_KO":r"CD19${}^{\mathrm{KO}}$",
}

for tumor in df_model.index.get_level_values("Tumor").unique():  
    yvals = df_model.loc[(tumor, mhc_levels_nalm6, xvals), "Ratio"].values
    ax.plot(xvals, yvals, lw=2.5, color=tumor_colors[tumor], label=nice_tumor_lbls[tumor], 
           ls=tumor_styles[tumor])
for side in ["top", "right"]:
    ax.spines[side].set_visible(False)
ax.set(ylabel=r"FC${}_{TCR \rightarrow CAR}$", 
        xlabel=r"TCR Antigen $\tau$ (s)")
ax.set_yscale("log", base=2)
#ax.set_ylim([0.48, 2.05])
ax.legend(fontsize=9, frameon=False, loc="upper left", bbox_to_anchor=(1.0, 1.0))
ax.axhline(1.0, ls="--", color="k", lw=1.0)
#fig.savefig(fig_dir + "antagonism_b16_surface_molecule_numbers.pdf", transparent=True, 
#            bbox_inches="tight")
plt.show()
plt.close()

# Check the CAR only response for each m, f (k=1)

In [ ]:
car_lrange = np.logspace(2, 6.2, 120)
car_responses = np.zeros([3*3, car_lrange.size])
i = 0
fms_list = list(itertools.product(range(1, 4), range(1, 4)))
cmaps_f = {1:sns.color_palette("Reds", 3), 
           2:sns.color_palette("Blues", 3), 
           3:sns.color_palette('Purples', 3)}
for (f, m) in fms_list:
    trial_kmf = (1, m, f)
    pvec_trial = np.asarray(all_results_dicts.get(str(trial_kmf)).get("param_estimates").get("MAP best"))
    res = repackage_tcr_car_params(pvec_trial, trial_kmf, *cost_args_loaded[:3])
    _, _, car_rates_trial, _, _, car_ri_trial, _, _, car_nmf_trial, _ = res
    for j in range(car_lrange.size):
        car_responses[i, j] = steady_akpr_i_1ligand(car_rates_trial, cd19_tau_l[0], 
                            car_lrange[j], car_ri_trial, car_nmf_trial)[car_nmf_trial[0]]
    i += 1

fig, ax = plt.subplots()
fig.set_size_inches(3, 3)
for tumor in cd19_levels_nalm6.keys():
    ax.axvline(cd19_levels_nalm6[tumor], ls="--", lw=1.0, color="grey")
    ax.annotate(tumor.strip("CD19_"), xy=(cd19_levels_nalm6[tumor], car_responses.max()), 
                va="bottom", ha="center", fontsize=6)
for i in range(len(fms_list)):
    f, m = fms_list[i]
    color = cmaps_f[f][m-1]
    ax.plot(car_lrange, car_responses[i], color=color,
            label=r"$m = {1}, f = {0}$".format(*fms_list[i]))
ax.set(xscale="log", yscale="log", xlabel=r"$L^C$" ,ylabel=r"$C^C_{N^C}$")
ax.legend(loc="upper left", bbox_to_anchor=(1, 1))
plt.show()
plt.close()

# Model predictions for AEBS CAR-T cells
APCs change, and also their respective TCR antigen. 

Healthy tissue, BEAS2B-GL lung cell line: 
 - Peptide: HHAT-WT
 - HLA-A2 level: a little over $10^6$
 - Her2 level: $2 \times 10^4$
 
Tumor cells, PC9-GL:
 - Peptide: HHAT-p8F
 - HLA-A2 level: $7 \times 10^4$
 - Her2 level: $4 \times 10^4$
 
Treat Her2-CAR affinity like CD19-CAR affinity to begin with, adjust if necessary. 

In [ ]:
# Nice big parameter dictionary
param_dict_her = {
    "car_rates": car_rates, 
    "car_ri": car_ri, 
    "car_nmf": car_nmf, 
    "all_rates": all_rates, 
    "tcr_nmf": tcr_nmf, 
    "ritot_vec": ritot_vec, 
    "nmf_both": nmf_both, 
    "tcr_thresh": tcr_thresh, # TCR threshold seems higher for PLCgamma? 
    "car_thresh": car_thresh
}

In [ ]:
# TODO: apply surface area correction factor?
df_her = pd.read_hdf(molec_counts_filename, key="surface_numbers_stats")
# The experiment only used BEAS2B_GL and PC9_GL
df_her = df_her.loc[df_her.index.isin(["BEAS2B", "PC9"], level="Cell")]
df_her = (df_her.rename({"BEAS2B":"BEAS2B_GL", "PC9":"PC9_GL"}, level="Cell", axis=0)
            .rename({"MHC":"HLA_A2"}, level="Marker", axis=0))
df_her

In [ ]:
# Data on L_T, L_C for both APC lines
size_factor_her = 1.0  # Unsure of the size, don't correct because qualitative anyways
mtc = "Geometric mean"
hla_levels = {
    "BEAS2B-GL": df_her.loc[("BEAS2B_GL", "HLA_A2"), mtc] / size_factor_her,
    "PC9-GL": df_her.loc[("PC9_GL", "HLA_A2"), mtc] / size_factor_her
}

her2_levels = {
    "BEAS2B-GL": df_her.loc[("BEAS2B_GL", "Her2"), mtc] / size_factor_her,
    "PC9-GL": df_her.loc[("PC9_GL", "Her2"), mtc] / size_factor_her
}

# High affinity ligand for the CAR, assume it is just like CD19.  
her2_tau = cd19_tau_l[0]

# Based on CD25 dose response EC50 conversion, tau for the two peptides
with open("data/pep_tau_map_others.json", "r") as h:
    other_taus = json.load(h)
hhat_pep_taus = {
    "HHAT-WT": other_taus.get("HHAT-WT"), 
    "HHAT-p8F": other_taus.get("HHAT-p8F")
}

# Which tumor has which peptide
tumor_peptide_map = {
    "BEAS2B-GL": "HHAT-WT", 
    "PC9-GL": "HHAT-p8F"
}

In [ ]:
tau_tcr_range = np.linspace(0.001, 8.0, 101)

df_model_aebs = pd.concat({t: compute_model_output_curve(tau_tcr_range, np.asarray([hla_levels[t]]),  
                (her2_tau, her2_levels[t]), param_dict_her, highlights=[hhat_pep_taus[tumor_peptide_map[t]]])
                    for t in hla_levels.keys()}, names=["Target"])

In [ ]:
# Plot results: just two antagonism ratio curves vs tau, highlight the peptides
# Aesthetic parameters
tumor_palette = {
    "BEAS2B-GL": "#CED5C3",  # pale green from fig. 5
    #"BEAS2B-GL": (167/255, 177/255, 112/255), # Slightly darker green (nucleus in fig. 5)
    #"PC9-GL": "#DFBCA8",  # pale pink from fig. 5
    "PC9-GL": (195/255, 156/255, 142/255),   # Slightly darker pink (nucleus in fig. 5)
}
peptide_palette = {
    "HHAT-WT": "b", 
    "HHAT-p8F": "r"
}
markerstyles = {
    "BEAS2B-GL":"X", 
    "PC9-GL": "o"
}
linestyles = {
    "BEAS2B-GL": "--", 
    "PC9-GL": "-"
}

fig, ax = plt.subplots()
fig.set_size_inches(2.75, 2.25)

peptide_labels = {
    "BEAS2B-GL": r"HHAT${}^{p8F}$ (Neoantigen)", 
    "PC9-GL": r"HHAT${}^{WT}$ (Self-antigen)", 
}
# Nicer name for tumors
tumor_labels = {
    "BEAS2B-GL": "BEAS-2B (Tissue)", 
    "PC9-GL": "PC9 (Tumor)"
}

ax.axhline(1.0, ls=":", lw=1.0, color="k")
for tumor in tumor_palette:
    xvals = np.sort(df_model_aebs.xs(tumor, level="Target").index
            .get_level_values(r"$\tau_T$").unique().values)
    yvals = df_model_aebs.loc[(tumor, slice(None), xvals), "Ratio"].values
    ax.plot(xvals, yvals, label=tumor_labels[tumor], color=tumor_palette[tumor], 
           ls=linestyles[tumor], alpha=1, lw=2.5)
    # highlight
    tau_mark = hhat_pep_taus[tumor_peptide_map[tumor]]
    mark_color = peptide_palette[tumor_peptide_map[tumor]]
    ymark = df_model_aebs.loc[(tumor, slice(None), tau_mark), "Ratio"].values
    yfact = 0.9 if tumor == "PC9-GL" else 1.0
    ax.plot([tau_mark], ymark, 
           ls="none", marker=markerstyles[tumor], mfc=mark_color, mec=mark_color, ms=8)
    ax.annotate(tumor_peptide_map[tumor], xy=(tau_mark+0.5, ymark*yfact), fontsize=10, 
                ha="left", va="center", color=mark_color)
for side in ["top", "right"]:
    ax.spines[side].set_visible(False)
ax.set_xlabel(r"Model $\tau_T$ (s)")
ax.set_ylabel(r"$\frac{\mathrm{Out(Her2 + HHAT)}}{\mathrm{Out(Her2)}}$", size=12)
ax.set_yscale("log", base=2)
#ax.set_ylim([0.48, 2.05])
ax.legend(fontsize=9, frameon=False, loc="lower left", bbox_to_anchor=(0.5, 0.1))
ax.axhline(1.0, ls="--", color="k", lw=1.0)
#fig.savefig(fig_dir + "antagonism_hhat_peptides_beas2b_pc9_numbers.pdf", transparent=True, 
#            bbox_inches="tight")
plt.show()
plt.close()

In [ ]:
df_model_aebs

In [ ]:
fname = "results/for_plots/model_predict_aebs_hhat_wt_neoantigen.h5"
df_model_save = df_model_aebs["Ratio"].copy()
df_model_save.index = df_model_save.index.set_names([a.strip("$").strip("\\") for a in df_model_save.index.names])
df_model_save.to_hdf(fname, key="df")

# Save all HHAT peptide parameters in a JSON file
hhat_params_dict = {
    "hla_levels": hla_levels, 
    "her2_levels": her2_levels, 
    "hhat_pep_taus": hhat_pep_taus, 
    "tumor_peptide_map": tumor_peptide_map,
    "her2_tau": her2_tau, 
    
}
fname = "results/for_plots/hhat_peptides_params_aebs_wt_neoantigen.json"
with open(fname, "w") as h:
    json.dump(hhat_params_dict, h, indent=2)